In [31]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve.meshes import MakeStructured2DMesh, MakeStructured3DMesh
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *
from netgen.geom2d import SplineGeometry
import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
import pandas as pd
import scipy
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt

In [32]:
def gen_latex_table(df, caption, label):
    """Generates a LaTeX table from a pandas DataFrame."""
    latex_table = df.to_latex(index=False, caption=caption, label=label, float_format="%.4e")
    return latex_table

In [33]:
def get_custom_frobenius_norm(A,theta):
   return A[0,0]**2 + A[1,1]**2 + (1/theta**2) * A[0,1]**2 + theta**2 * A[1,0]**2


In [34]:
## get problem parameters and geometry
problem = problems.problem4
phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 0.05
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6
gel_measures = [[15.0,90,1.62],[23.5,90,3.0]]

G = Parameter(G_target)

def reference_energy(lambda1,lambda2,lambda3):
    G=G_target
    nu = KBTV
    J= lambda1*lambda2*lambda3
    return 0.5*G*(lambda1**2 + lambda2**2 + lambda3**2) + nu*((J-phi0)*np.log(1-phi0/J) + phi0*chi*(1-phi0/J))

def dH( J):
        return phi0/J + np.log(1-phi0/J) + chi * (phi0/J)**2
def aux_isotropic(s):
    return s*dH(s*s*s) + G_target/KBTV

l_iso = (scipy.optimize.fsolve(aux_isotropic, 1.7))[0]

reference_energy_val = reference_energy(l_iso,l_iso,l_iso)
def H_scalar(J):
        return (J - phi0) * np.log(1-phi0/J) + phi0*chi*(1-phi0/J)
def lambda_uni(lamb):
        return G_target*0.5*(2+lamb**2)+KBTV*H_scalar(lamb)
    
l_uni = minimize_scalar(lambda_uni,bounds=(0,2),method='bounded').x
def auxFunctionUniaxial(s):
    gamma = G_target/(KBTV)
    return dH(s) +gamma*s

def run_experiment(delta,measures,n_delta,h = 0.05):
    delta = np.round(delta,2)
    w,l,d = measures
    theta = w/l
    rect = SplineGeometry()

    pnts = [(delta/2,-1/2), (0.5,-1/2), (0.5,1/2), (delta/2,1/2)]

    p1 = rect.AppendPoint(*pnts[0], hpref=1) # to close the rectangle
    p2 = rect.AppendPoint(*pnts[1], hpref=1) # to close the rectangle
    p3 = rect.AppendPoint(*pnts[2], hpref=1) # to close the rectangle
    p4 = rect.AppendPoint(*pnts[3], hpref=1) # to close the rectangle

    curves = [[["line",p1,p2],"bottom"],
            [["line",p2,p3],"right"],
            [["line",p3,p4],"top"],
            [["line",p4,p1],"left"]]


    rect.Append(curves[0][0], bc = curves[0][1], leftdomain=1, rightdomain=0)
    rect.Append(curves[1][0], bc = curves[1][1], leftdomain=1, rightdomain=0) #0.03
    rect.Append(curves[2][0], bc = curves[2][1], leftdomain=1, rightdomain=0) #0.03
    rect.Append(curves[3][0], bc = curves[3][1], leftdomain=1, rightdomain=0) #0.05

    mapping_first_mesh = lambda x,y: ((x*(0.5 - delta/2)) +delta/2, y-0.5)
    mesh = MakeStructured2DMesh(
        nx=19-n_delta, ny=int(1/h), mapping=mapping_first_mesh, quads=False)
    Draw(mesh)

    def Gel_energy_functional(u,beta):
            Grad_custom = CF((Grad(u)[0,0], (1/theta)*Grad(u)[0,1],
                            theta*Grad(u)[1,0], Grad(u)[1,1]),dims=(2,2))
            F = Id(2) + Grad_custom
            J = Det(F) * beta
            phi = phi0/J
            H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi)
            return 0.5*(G)* (get_custom_frobenius_norm(F,theta) + beta**2) + H * KBTV 
    def energy_functional(F):
        J = Det(F)
        C = F.trans* F
    
        G = G_target
      
        H = lambda J: (J - phi0) * log(1-phi0/J) + phi0*chi*(1-phi0/J)
        
        return 0.5*G*(Trace(C)) + KBTV * H(J) -  reference_energy_val


    ## Generate spaces and forms
    """
    To bond the gel go to geometries and describe the bonding
    face there, not here.
    """
    fesu = VectorH1(mesh, order = ord, dirichlet = "left")
    fesbeta = H1(mesh, order = ord)
    fes = FESpace([fesu, fesbeta])
    u, beta = fes.TrialFunction()
    BF = BilinearForm(fes)
    gfu = GridFunction(fes)
    gfu.components[0].Set((0,0))
    gfu.components[1].Set(1)
    BF += Variation(Gel_energy_functional(u,beta).Compile()*dx)
    ##### solve with newton
    max_iter_newton = 100
    newton_damp = 0.1
    softening_n = 150

    tol_newton = 1e-8
    gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))

    for num in range(len(gammas)):
        G.Set(gammas[num])
        res = gfu.vec.CreateVector()
        w_vec = gfu.vec.CreateVector()
        for iter in range(max_iter_newton):
            with TaskManager():
                BF.Apply(gfu.vec,res)
                BF.AssembleLinearization(gfu.vec)
                inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                w_vec.data = newton_damp * inv * res
            gfu.vec.data += -w_vec
            if abs(InnerProduct(w_vec,res)) < tol_newton:
                break 

    gfu2 = GridFunction(fesu)
    gfu2.components[0].Set(gfu.components[0].components[1])
    gfu2.components[1].Set(gfu.components[0].components[0])

    
    def H_scalar(J):
        return (J - phi0) * np.log(1-phi0/J) + phi0*chi*(1-phi0/J)

    def lambda_uni(lamb):
        return G_target*0.5*(2+lamb**2)+KBTV*H_scalar(lamb)
    
    l_uni = minimize_scalar(lambda_uni,bounds=(0,4),method='bounded').x
    h = 1.0
    mapping_unit_cube = lambda x,y,z: ((1/2)*x,y-0.5 , z)
    mapping_phisical_gel = lambda x ,y ,z: (x*l/2, -w/2 + y*w, z*d)

    mesh_unit_cube = MakeStructured3DMesh(nx=200, ny=int(w/h), nz=int(d/h),mapping=mapping_unit_cube, hexes = False)
    mesh_phisical_gel = MakeStructured3DMesh(
        nx=200, ny=int(w/h), nz=int(d/h),
        mapping=mapping_phisical_gel, hexes= False)
    
    
    mesh_center = MakeStructured3DMesh(
        nx=200, ny=int(w/h), nz=int(d/h),
        mapping=lambda x,y,z: (-l*delta/2 + x*l*delta, -w/2 + y*w, z*d), hexes= False)
    
    mesh_wing_cube = MakeStructured3DMesh(
        nx=200, ny=int(w/h), nz=int(d/h),
        mapping=lambda x,y,z: (delta/2 + x*(1-delta)/2, -1/2 + y, z), hexes= False)
    
    aux_z = IfPos(-delta/2+x, z,0)
    
    # create the gridfunctions for the different meshes
    fes_unit_cube = VectorH1(mesh_wing_cube, order=ord)
    gfu_unit_cube = GridFunction(fes_unit_cube)
    gfu_unit_cube.components[0].Set(gfu.components[0].components[0])
    gfu_unit_cube.components[1].Set(gfu.components[0].components[1])
    gfu_unit_cube.components[2].Set(z * gfu.components[1]- aux_z)
    
    fes_center = VectorH1(mesh_center, order=ord)   
    gfu_center = GridFunction(fes_center)
    gfu_center.components[2].Set((l_uni-1) * z)

    fesu_3d = VectorH1(mesh_unit_cube, order=ord)
    gfu3d = GridFunction(fesu_3d)
    gfu3d.components[0].Set(gfu.components[0].components[0])
    gfu3d.components[1].Set(gfu.components[0].components[1])
    gfu3d.components[2].Set(z*gfu.components[1]-aux_z)
    
    phisical_gel_fes = VectorH1(mesh_phisical_gel, order=ord)
    phisical_gel_gfu = GridFunction(phisical_gel_fes)
    phisical_gel_gfu.components[0].vec.data = l*gfu3d.components[0].vec
    phisical_gel_gfu.components[1].vec.data = w*gfu3d.components[1].vec
    phisical_gel_gfu.components[2].vec.data = d*gfu3d.components[2].vec

    gfu_aux = GridFunction(phisical_gel_fes)
    gfu_aux.components[2].Set(IfPos(delta*l/2-x, (l_uni-1)*z,0))
    gfu_final = gfu_aux + phisical_gel_gfu

    with TaskManager():
         # calculate (a)
        a_measure = gfu_final(mesh_phisical_gel(l/2,w/2,0))[1] - gfu_final(mesh_phisical_gel(l/2,-w/2,0))[1] + w 
        # calculate (b)
        b_measure = w # direct since uniaxial deformation in this section.
        # calculate (c)
        c_measure = 2 * (gfu_final(mesh_phisical_gel(l/2,0,0))[0]+(1-delta)*l/2) + l*delta
        # calculate (d)
        d_measure = gfu_final(mesh_phisical_gel(l/2,0,d))[2]
        # calculate (e)
        e_measure = gfu_final(mesh_phisical_gel(0,0,d))[2] 

        F_center = Id(3) + Grad(gfu_center)
        F_center_wing = Id(3) + Grad(gfu_final)
        energy_center = GridFunction(H1(mesh_phisical_gel, order=1))
        energy_density_center_wing = GridFunction(H1(mesh_phisical_gel, order=1))
        energy_center.Set(energy_functional(F_center))
        energy_density_center_wing.Set(energy_functional(F_center_wing))
        total_energy = 2 * Integrate(energy_density_center_wing,mesh_phisical_gel,order=5) 
    # calculate average energy
    avg_energy = total_energy/(l*w*d)
   
    
    # generate plot and save it 
    # show the linear regression 
    return {
        'delta': delta,
        'a': a_measure,
        'b': b_measure,
        'c': c_measure,
        'd': d_measure,
        'e': e_measure,
        'total_energy': total_energy,
        'avg_energy': avg_energy*1000
        }, gfu_final, mesh_phisical_gel


In [ ]:
def run_all_experiments(deltas, measures):
    for measure in measures:
        print(f"Running experiments for measures: {measure}")
        results = []
        for n_delta, delta in enumerate(deltas):
            res, gfu_final, mesh_phisical_gel = run_experiment(delta, measure,n_delta ,h=0.1)
            results.append(res)
            print(f"Delta: {delta}, Measures: {measure}, Results: {res}")
        results = pd.DataFrame(results)
        caption = f"Results for measures {measure} with varying delta"
        label = f"tab:results_{measure[0]}_{measure[1]}_{measure[2]}"
        latex_table = gen_latex_table(results, caption, label)
        # save table
        with open(f"results_{measure[0]}_{measure[1]}_{measure[2]}.tex", "w") as f:
            f.write(latex_table)    
        deltas = results['delta'].values
        avg_energy = results['total_energy'].values
        # get slope
        slope, intercept = np.polyfit(deltas[:-5], avg_energy[:-5], 1)
        #plot polyfit
        plt.figure(figsize=(10, 6))
        plt.plot(deltas, avg_energy, marker='o', label='Average Energy')
        x = np.linspace(0,1,100)
        plt.plot(x, slope * x + intercept, label='Linear Fit', linestyle='--')
        plt.title('Average Energy vs Delta with Linear Fit')
        plt.xlabel('Delta')
        plt.ylabel('Average Energy')
        plt.savefig(f"average_energy_vs_delta_{measure[0]}_{measure[1]}_{measure[2]}.png")
        print(f"Slope: {slope}, Intercept: {intercept}")
run_all_experiments(np.arange(0.05,1,0.05), gel_measures)
        

Running experiments for measures: [15.0, 90, 1.62]


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Latex resumen del experimento
hacer para w, d = 23.5 /15, 3, 1.62
subir foto a overleaf
reportar resultados actuales.

cambiar a make structrured mesh
mostrar algunas mallas en paraview
comparar foto contra ecuación 24 y 25 del paper (último) y figuras de cortes transversales (fig 11)
comparar abcde  modelo reducido y 3D (pedir las soluciones) actualizar github :) 

In [20]:
import numpy as np
x = np.arange(0.05,1,0.05)
print(x)

[0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7
 0.75 0.8  0.85 0.9  0.95]
